<a href="https://colab.research.google.com/github/ssrbazpur/Data-Science/blob/master/Data%20Science%20project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas 
month2 = pandas.read_pickle("/content/drive/My Drive/Data Science/m2.pkl") 
month1 = pandas.read_pickle("/content/drive/My Drive/Data Science/m1.pkl")

In [0]:
import dask.dataframe as dd

In [0]:
m2 = dd.from_pandas(month2, npartitions=8)
m1 = dd.from_pandas(month1, npartitions=8)

In [0]:
m3=dd.concat([m1,m2,m1,m2,m1,m2],axis=0,interleave_partitions=True)

In [7]:
!pip install --upgrade google-cloud-storage
!pip install --upgrade google-cloud-bigquery[bqstorage,pandas]

     |████████████████████████████████| 81kB 2.3MB/s 
     |████████████████████████████████| 81kB 6.9MB/s 
     |████████████████████████████████| 71kB 9.7MB/s 
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.2 which is incompatible.
ERROR: google-cloud-bigquery 1.21.0 has requirement google-resumable-media!=0.4.0,<0.5.0dev,>=0.3.1, but you'll have google-resumable-media 0.5.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: google-resumable-media 0.4.1
    Uninstalling google-resumable-media-0.4.1:
      Successfully uninstalled google-resumable-media-0.4.1
  Found existing installation: google-api-core 1.15.0
    Uninstalling google-api-core-1.15.0:
      Successfully uninstalled google-api-core-1.15.0
  Found existing installation: google-cloud-core 1.0.3
    Uninstalling google-cloud-core-1.0.3:
  

     |████████████████████████████████| 163kB 2.5MB/s 
     |████████████████████████████████| 61kB 3.5MB/s 
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.14.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: six 1.12.0
    Uninstalling six-1.12.0:
      Successfully uninstalled six-1.12.0
  Found existing installation: google-cloud-bigquery 1.21.0
    Uninstalling google-cloud-bigquery-1.21.0:
      Successfully uninstalled google-cloud-bigquery-1.21.0


In [8]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/drive/My Drive/Data Science/NYC Taxi-372c8c9c5c4e.json"
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [0]:
project_id = 'nyc-taxi-265120' #@param{type:"string"}
from google.cloud import bigquery
#from google.cloud import bigquery_storage_v1beta1
# Create BigQuery client
client = bigquery.Client(project = project_id)
# Create BigQuery storage client
#bq_storage_client = bigquery_storage_v1beta1.BigQueryStorageClient()

In [0]:
taxidemand = """
  
    SELECT 
        mo,da,temp,rain_drizzle,
        cast(year as STRING) as year,
        EXTRACT (DAYOFYEAR FROM CAST(CONCAT(year,'-',mo,'-',da) AS TIMESTAMP)) AS daynumber,
    FROM `bigquery-public-data.noaa_gsod.gsod*`
    WHERE year='2015' and stn='725030'  AND _TABLE_SUFFIX between '2014' and '2016' --station id 725030=LaGuardia 
   

  

"""
data = client.query(taxidemand).to_dataframe()

In [11]:
data.head()


,mo,da,temp,rain_drizzle,year,daynumber
0,01,01,31.2,0,2015,1
1,01,02,38.6,0,2015,2
2,01,13,30.2,1,2015,13
3,01,14,22.0,0,2015,14
4,01,21,30.5,0,2015,21


In [0]:
weather = dd.from_pandas(data, npartitions=8)

In [13]:
m3.head()

KeyboardInterrupt: ignored

In [0]:
yellow=m3[['pickup_datetime', 'pickup_latitude', 'pickup_longitude']]
yellow.columns=['Date', 'Lat', 'Lon'] # rename for future merge

In [15]:

yellow['year'] = yellow['Date'].dt.year.astype(int)
yellow['day'] = yellow['Date'].dt.day.astype(int)
yellow['month'] = yellow['Date'].dt.month.astype(int)
yellow.head()

,Date,Lat,Lon,year,day,month
0,2015-01-25 16:48:51+00:00,40.734119,-74.004372,2015,25,1
1,2015-01-02 11:55:25+00:00,40.747742,-73.974052,2015,2,1
2,2015-01-10 12:46:52+00:00,40.727013,-74.003326,2015,10,1
3,2015-01-09 18:31:47+00:00,40.741936,-73.980904,2015,9,1
4,2015-01-04 00:37:01+00:00,40.729877,-74.000725,2015,4,1


In [0]:
weather['year']=weather['year'].astype(int)
weather['mo']=weather['mo'].astype(int)
weather['da']=weather['da'].astype(int)

In [0]:
df2 = yellow.merge(weather, how='left', left_on=['year','month','day'], right_on=['year','mo','da'])

In [24]:
df2.head()

,Date,Lat,Lon,year,day,month,mo,da,temp,rain_drizzle,daynumber
0,2015-01-17 02:49:20+00:00,40.743351,-73.973473,2015,17,1,1,17,22.3,0,17
1,2015-01-17 17:31:31+00:00,40.758602,-73.981613,2015,17,1,1,17,22.3,0,17
2,2015-01-17 12:04:24+00:00,40.771538,-73.990471,2015,17,1,1,17,22.3,0,17
3,2015-01-17 16:33:13+00:00,40.786850,-73.972870,2015,17,1,1,17,22.3,0,17
4,2015-01-17 17:50:48+00:00,40.735325,-73.985992,2015,17,1,1,17,22.3,0,17


In [18]:
!pip install dask[dataframe]

In [0]:
df_agg_graph = df2.groupby(['daynumber','Lat','Lon'])['Lat'].count()
df_agg_graph=df2.groupby( [ 'daynumber','Lat','Lon'] ).size().to_frame(name = 'count').reset_index()

In [0]:
weatherinfo=df2.groupby( [ 'daynumber','temp','rain_drizzle'] )['Lat'].size().to_frame(name = 'count').reset_index()

In [34]:
df_agg_graph.head()

,daynumber,Lat,Lon,count
0,17,0.000000,0.000000,30042
1,17,30.191471,-81.364006,3
2,17,30.589655,-77.957985,6
3,17,33.127785,-81.872124,3
4,17,37.511635,-65.930099,3


In [38]:
weatherinfo.head()

,daynumber,temp,rain_drizzle,count
0,17,22.3,0,1430481
1,44,17.6,0,1458723
2,49,24.2,0,1309821
3,10,21.7,0,1546620
4,14,22.0,0,1327968


In [0]:
final_data = df_agg_graph.merge(weatherinfo, how='left', left_on=['daynumber'], right_on=['daynumber'])

In [2]:
final_data.head()

NameError: ignored

In [0]:
final_data = final_data.drop('count_y', axis=1)

In [45]:
len(final_data) 

24255078

In [0]:
X=final_data[['daynumber','Lat','Lon','temp','rain_drizzle']]
y=final_data[['count_x']]


In [0]:
X.to_parquet('/content/drive/My Drive/Data Science/X.parquet')
y.to_parquet('/content/drive/My Drive/Data Science/y.parquet')

In [0]:
!pip install dask_glm
!pip install dask-ml --upgrade

In [40]:
!pip install -U ipykernel

     |████████████████████████████████| 122kB 2.8MB/s 
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.6.0, but you'll have ipykernel 5.1.3 which is incompatible.
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.14.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement tornado~=4.5.0, but you'll have tornado 6.0.3 which is incompatible.
  Found existing installation: ipykernel 4.6.1
    Uninstalling ipykernel-4.6.1:
      Successfully uninstalled ipykernel-4.6.1


In [38]:
!sudo pip install dklearn

ERROR: Could not find a version that satisfies the requirement dklearn (from versions: none)
ERROR: No matching distribution found for dklearn


In [46]:
!pip install dask

In [48]:
!pip install dask distributed

In [51]:
!pip install dask[complete] distributed --upgrade

     |████████████████████████████████| 788kB 2.8MB/s 
Requirement already up-to-date: distributed in /usr/local/lib/python3.6/dist-packages (2.9.3)
  Found existing installation: dask 2.9.1
    Uninstalling dask-2.9.1:
      Successfully uninstalled dask-2.9.1


In [56]:
!pip install dask distributed --upgrade

Requirement already up-to-date: dask in /usr/local/lib/python3.6/dist-packages (2.9.2)
Requirement already up-to-date: distributed in /usr/local/lib/python3.6/dist-packages (2.9.3)


In [62]:
!pip install fastparquet

     |████████████████████████████████| 153kB 2.7MB/s 
     |████████████████████████████████| 61kB 5.6MB/s 
  Created wheel for fastparquet: filename=fastparquet-0.3.2-cp36-cp36m-linux_x86_64.whl size=253144 sha256=60099fc022b6c56a0e688f522cf0efad58a336e7e1c07c4b6fd772433746ce00
  Stored in directory: /root/.cache/pip/wheels/b9/36/13/01416a760ddcab0eb8281ec9c9ffcbed945c9b831647c8b904
  Created wheel for thrift: filename=thrift-0.13.0-cp36-cp36m-linux_x86_64.whl size=345264 sha256=73bc6e65b169fceca1c4bb4fa7c4bccb30ad6eefed30ea0a1eb73bfcac354654
  Stored in directory: /root/.cache/pip/wheels/02/a2/46/689ccfcf40155c23edc7cdbd9de488611c8fdf49ff34b1706e
Successfully built fastparquet thrift


In [0]:
import pandas as pd
X=pd.read_parquet('/content/drive/My Drive/Data Science/X.parquet', engine='fastparquet')
y=pd.read_parquet('/content/drive/My Drive/Data Science/y.parquet', engine='fastparquet')

In [0]:
X.head()

In [0]:
y.head()